<a href="https://colab.research.google.com/github/maikCyphlock/Mylab/blob/main/whisper-faslk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/sanchit-gandhi/whisper-jax.git

!pip install flask-ngrok


In [ ]:
# Descarga el archivo
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19879 sha256=04a1754051133babe321541fbbf0234715c680132ff8ef713f4eb2577282b105
  Stored in directory: /root/.cache/pip/wheels/31/49/9c/44b13823eb256a3b4dff34b972f7a3c7d9910bfef269e59bd7
Successfully built pyngrok


In [ ]:
from flask import Flask, jsonify, request
import requests
from whisper_jax import FlaxWhisperPipline
from pyngrok import ngrok, conf


# Construct the API client
# client = ngrok.Client("2D91FFQtqYkkGgUnospalrROWRd_2aG5upiFWGmQCZqKLqpeG")
app = Flask(__name__)
conf.API_KEY = "2D91FFQtqYkkGgUnospalrROWRd_2aG5upiFWGmQCZqKLqpeG"
# instantiate pipeline
pipeline = FlaxWhisperPipline("openai/whisper-small")

# define route to handle POST requests
@app.route('/transcribe', methods=['POST'])
def transcribe():
    # check if request has an audio URL
    if 'url' not in request.json:
        return jsonify({'error': 'Missing URL parameter'}), 400

    # download audio file from URL
    url = request.json['url']
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return jsonify({'error': str(e)}), 400

    # save audio file to disk
    with open('audio.mp3', 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)

    # transcribe audio file using pipeline
    text = pipeline('./audio.mp3')

    return jsonify({'text': text}), 200

if __name__ == '__main__':
    public_url = ngrok.connect(5000).public_url
    print(f"Ngrok URL: {public_url}")
    app.run(port=5000,debug=True)


Ngrok URL: https://0e39-35-237-111-158.ngrok.io
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
